In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

In [3]:
colors = [
    "rgba(83, 135, 221, 1)",
    "rgba(34, 148, 135, 1)",
    "rgba(218,76,76, 1)",
    "rgba(237, 183, 50, 1)",
]

In [ ]:
import re

def plot_metrics(
    csv_path,
    output_path,
    x_col="Step",
    pipelines=None,
    pipeline_names=None,
    colors=None,
    metric="lpips",
    title=None,
    y_label=None,
    report_final_values=True,
    final_step=30000,
    figure_size=(None, None)  # (width, height)
):
    """
    Plot metrics from a CSV file with customizable pipelines, colors, and labels.
    
    Parameters:
    -----------
    csv_path : str or Path
        Path to the CSV file containing the data
    output_path : str or Path
        Path to save the output figure
    x_col : str, default="Step"
        Column name for x-axis values
    pipelines : list of str, default=None
        List of pipeline identifiers to plot. If None, detect all pipelines.
    pipeline_names : dict, default=None
        Dictionary mapping pipeline identifiers to display names
    colors : dict or list, default=None
        Dictionary mapping pipeline identifiers to colors, or list of colors to use in order
    metric : str, default="lpips"
        Metric to plot (e.g., "lpips", "psnr")
    title : str, default=None
        Title for the plot. If None, use capitalized metric name.
    y_label : str, default=None
        Label for y-axis. If None, use capitalized metric name.
    report_final_values : bool, default=True
        Whether to print final values for each pipeline
    final_step : int, default=30000
        Step value to use for reporting final values
    figure_size : tuple, default=(None, None)
        Width and height of the figure
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        The generated figure
    """
    csv_path = Path(csv_path)
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    df = pd.read_csv(csv_path)
    df = df.astype(float)
    df[x_col] = df[x_col].astype(int)
    
    steps = df[x_col].values
    
    # Set up title and y-label
    if title is None:
        title = metric.upper()
    if y_label is None:
        y_label = metric.upper()
    
    if pipelines is None:
        pattern = r'pipeline: ([^-]+)'
        all_pipelines = set()
        for col in df.columns:
            match = re.search(pattern, col)
            if match:
                all_pipelines.add(match.group(1))
        pipelines = list(all_pipelines)
    
    if pipeline_names is None:
        pipeline_names = {p: p for p in pipelines}
    
    # Set up colors
    if colors is None:
        default_colors = [
            "rgba(83, 135, 221, 1)",  # Blue
            "rgba(218, 76, 76, 1)",   # Red
            "rgba(75, 192, 192, 1)",  # Teal
            "rgba(153, 102, 255, 1)", # Purple
            "rgba(255, 159, 64, 1)",  # Orange
        ]
        if isinstance(colors, list):
            colors = {p: default_colors[i % len(default_colors)] for i, p in enumerate(pipelines)}
        else:
            colors = {p: default_colors[i % len(default_colors)] for i, p in enumerate(pipelines)}
    elif isinstance(colors, list):
        colors = {p: colors[i % len(colors)] for i, p in enumerate(pipelines)}
    
    fig = go.Figure()
    
    # For each pipeline, add traces
    for i, pipeline in enumerate(pipelines):
        # Select columns for this pipeline
        pipeline_cols = [col for col in df.columns if f"pipeline: {pipeline} " in col and metric in col.lower()]
        
        # Find mean, min, max columns
        min_column = [col for col in pipeline_cols if col.endswith("__MIN")]
        max_column = [col for col in pipeline_cols if col.endswith("__MAX")]
        mean_column = [col for col in pipeline_cols if not col.endswith("__MIN") and not col.endswith("__MAX")]
        
        if not mean_column:
            print(f"Warning: No data found for pipeline '{pipeline}' with metric '{metric}'")
            continue
        
        mean_vals = df[mean_column].values.flatten()
        print(f"{pipeline}: {mean_vals[-1]:.4f}")
        fig.add_trace(go.Scatter(
            x=steps,
            y=mean_vals,
            mode='lines',
            name=pipeline_names.get(pipeline, pipeline),
            line=dict(width=6, color=colors[pipeline]),
        ))
        
        # Add shaded area for min-max if available
        if min_column and max_column:
            min_vals = df[min_column].values.flatten()
            max_vals = df[max_column].values.flatten()
            
            fig.add_trace(go.Scatter(
                x=np.concatenate([steps, steps[::-1]]),
                y=np.concatenate([min_vals, max_vals[::-1]]),
                fill='toself',
                fillcolor=colors[pipeline].replace("1)", "0.2)"),
                line=dict(color='rgba(255,255,255,0)'),
                hoverinfo="skip",
                showlegend=False
            ))

    
    layout_args = dict(
        font={'size': 25},
        title={
            'text': title,
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title=x_col,
        yaxis_title=y_label,
        legend={
            'yanchor': 'bottom',
            'y': 0.8 if metric == "lpips" else 0.1,
            'xanchor': 'right',
            'x': 1.0
        },
        template='plotly_white',
    )
    
    if figure_size[0]:
        layout_args['width'] = figure_size[0]
    if figure_size[1]:
        layout_args['height'] = figure_size[1]
    
    fig.update_layout(**layout_args)
    
    fig.write_image(output_path)
    return fig


fig = plot_metrics(
    csv_path="../../artifacts/wandb_csvs/all_lpips.csv",
    output_path="../../artifacts/figures/lpips_rgbd_colmap.pdf",
    # output_path="../../artifacts/figures/lpips_rgbd_colmap.pdf",
    pipelines=[
        'rgbd2colmap_no_icp',
        # 'rgbd2colmap_colmap_poses',
        'colmap_reconstruction',
        # 'rgbd2colmap'
        ],
    pipeline_names={
        'rgbd2colmap_no_icp': 'RGBD',
        # 'rgbd2colmap_colmap_poses': 'RGBD + COLMAP Poses',
        'colmap_reconstruction': 'COLMAP'
        # 'rgbd2colmap': 'With ICP'
    },
    colors={
        'rgbd2colmap_no_icp': "rgba(83, 135, 221, 1)",  # Blue
        # 'rgbd2colmap_colmap_poses': "rgba(34, 148, 135, 1)",  # Teal
        'colmap_reconstruction': "rgba(218, 76, 76, 1)"  # Red,
        # 'rgbd2colmap': "rgba(237, 183, 50, 1)"  # Yellow
    },
    metric="lpips",
    title="LPIPS",
    figure_size=(800, 500)
)
fig.show()

# # For PSNR
fig = plot_metrics(
    csv_path="../../artifacts/wandb_csvs/all_psnrs.csv",
    output_path="../../artifacts/figures/psnr_rgbd_colmap.pdf",
    pipelines=[
        'rgbd2colmap_no_icp',
        # 'rgbd2colmap_colmap_poses',
        'colmap_reconstruction',
        # 'rgbd2colmap'
        ],
    pipeline_names={
        'rgbd2colmap_no_icp': 'RGBD',
        # 'rgbd2colmap_colmap_poses': 'COLMAP Poses',
        'colmap_reconstruction': 'COLMAP',
        # 'rgbd2colmap': 'No ICP'
    },
    colors={
        'rgbd2colmap_no_icp': colors[0],  # Blue
        # 'rgbd2colmap_colmap_poses': colors[1],  # Teal
        'colmap_reconstruction': colors[2]  # Red,
        # 'rgbd2colmap': colors[3]  # Yellow
    },
    metric="psnr",
    title="PSNR",
    figure_size=(800, 500),
)
fig.show()

rgbd2colmap_no_icp: 0.1667
colmap_reconstruction: 0.1602


rgbd2colmap_no_icp: 30.8743
colmap_reconstruction: 31.9757


In [ ]:
from typing import List, Dict, Union, Tuple, Optional

def plot_bar_comparison(
    csv_path: str,
    output_path: str,
    data_key: str,
    pipelines: List[Tuple[str, str, str]] = None,
    title: str = None,
    y_label: str = None,
    y_scale: str = 'linear',
    y_tickvals: List[float] = None,
    y_ticktext: List[str] = None,
    figure_size: Tuple[Optional[int], Optional[int]] = (None, None),
    conversion_factor: float = 1.0,
    print_stats: bool = True
):
    """
    Create a bar plot comparing different pipelines from CSV data.
    
    Parameters:
    -----------
    csv_path : str or Path
        Path to the CSV file containing the data
    output_path : str or Path
        Path to save the output figure
    data_key : str
        Column key for the data to plot (e.g., 'peak_memory_kb', 'reconstruction_elapsed_time')
    pipelines : list of tuples, default=None
        List of (pipeline_id, color, label) tuples for the pipelines to include.
        If None, will include all pipelines found in the data.
    title : str, default=None
        Title for the plot. If None, use a generic title based on data_key.
    y_label : str, default=None
        Label for y-axis. If None, use data_key with spaces.
    y_scale : str, default='linear'
        Scale for y-axis ('linear' or 'log')
    y_tickvals : list of float, default=None
        Custom tick values for y-axis
    y_ticktext : list of str, default=None
        Custom tick labels corresponding to y_tickvals
    figure_size : tuple, default=(None, None)
        Width and height of the figure
    conversion_factor : float, default=1.0
        Factor to convert data values (e.g., 1/1024 to convert KB to MB)
    print_stats : bool, default=True
        Whether to print summary statistics
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        The generated figure
    """
    csv_path = Path(csv_path)
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    df = pd.read_csv(csv_path)
    
    if "Step" in df.columns:
        df = df.drop(columns=["Step"])
    df = df.astype(float)
    df = df.loc[:, ~df.columns.str.contains("MIN|MAX")]
    
    rows = []
    for full_key, value_series in df.items():
        scene_pipeline_info = full_key.replace(f' - {data_key}', '')
        parts = scene_pipeline_info.split('-')
        
        if len(parts) >= 3:  # Format: scene-pipeline-timestamp
            scene = parts[0]
            pipeline = parts[1]
        elif len(parts) == 2:  # Format: pipeline-timestamp or scene-pipeline
            if 'pipeline:' in parts[0]:
                scene = 'unknown'
                pipeline = parts[0].split('pipeline:')[1].strip()
            else:
                scene = parts[0]
                pipeline = parts[1]
        else:
            scene = 'unknown'
            pipeline = parts[0]
        
        # Apply conversion factor
        value = value_series.iloc[0] * conversion_factor
        
        rows.append({'scene': scene, 'pipeline': pipeline, data_key: value})
    
    parsed_df = pd.DataFrame(rows)
    
    grouped = parsed_df.groupby("pipeline")[data_key]
    summary = grouped.agg(['mean', 'std']).reset_index()
    
    if pipelines is None:
        default_colors = [
            "rgba(83, 135, 221, 1)",  # Blue
            "rgba(218, 76, 76, 1)",   # Red
            "rgba(75, 192, 192, 1)",  # Teal
            "rgba(153, 102, 255, 1)", # Purple
            "rgba(255, 159, 64, 1)",  # Orange
        ]
        pipelines = [(p, default_colors[i % len(default_colors)], p) 
                     for i, p in enumerate(summary['pipeline'].unique())]
    
    fig = go.Figure()
    
    for pipeline_id, color, label in pipelines:
        subset = summary[summary['pipeline'] == pipeline_id]
        
        if subset.empty:
            print(f"Warning: No data found for pipeline '{pipeline_id}'")
            continue
        
        fig.add_trace(go.Bar(
            x=[label],
            y=subset['mean'],
            name=label,
            error_y=dict(
                type='data',
                array=subset['std'],
                visible=True,
                color='rgb(50, 50, 50)',
                thickness=6
            ),
            marker_color=color
        ))
        
        if print_stats:
            print(f"{pipeline_id}: {subset['mean'].iloc[0]:.2f} ± {subset['std'].iloc[0]:.2f}")
    
    if title is None:
        title = f"{data_key.replace('_', ' ').title()} Comparison"
    
    if y_label is None:
        y_label = data_key.replace('_', ' ').title()
    
    yaxis_config = dict(
        type=y_scale,
        tickangle=0,
        automargin=True,
        nticks=6 if y_tickvals is None else len(y_tickvals)
    )
    
    if y_tickvals is not None:
        yaxis_config['tickvals'] = y_tickvals
    
    if y_ticktext is not None:
        yaxis_config['ticktext'] = y_ticktext
    
    layout_args = dict(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center',
        ),
        xaxis_title="Pipeline",
        yaxis_title=y_label,
        font=dict(size=25),
        legend=dict(
            yanchor='bottom',
            y=0.8,
            xanchor='left' if data_key == "reconstruction_elapsed_time" else 'right',
            x=0.0 if data_key == "reconstruction_elapsed_time" else 1.0,
        ),
        margin=dict(b=100),
        yaxis=yaxis_config,
        template='plotly_white'
    )
    
    if figure_size[0]:
        layout_args['width'] = figure_size[0]
    if figure_size[1]:
        layout_args['height'] = figure_size[1]
    
    fig.update_layout(**layout_args)
    
    fig.write_image(output_path)
    return fig

# Example 1: Reconstruction time
fig1 = plot_bar_comparison(
    csv_path="../../artifacts/wandb_csvs/no_icp_rgbd_recon_time.csv",
    output_path="../../artifacts/figures/recon_time_icp_no_icp.pdf",
    data_key="reconstruction_elapsed_time",
    pipelines=[
        ('rgbd2colmap_no_icp', colors[0], 'No ICP'),
        # ('rgbd2colmap_colmap_poses', colors[1], 'RGBD + COLMAP Poses'),
        # ('colmap_reconstruction', colors[2], 'COLMAP'),
        ('rgbd2colmap', colors[3], 'With ICP')
    ],
    title="Initialization Reconstruction Time",
    y_label="Time (s)",
    y_scale="log",
    y_tickvals=[5, 10, 20, 40, 80, 160],
    figure_size=(800, 500)
)
fig1.show()

# Example 2: Memory usage
fig2 = plot_bar_comparison(
    csv_path="../../artifacts/wandb_csvs/no_icp_rgbd_recon_mem.csv",
    output_path="../../artifacts/figures/memory_usage_icp_no_icp.pdf",
    data_key="peak_memory_kb",
    pipelines=[
        ('rgbd2colmap_no_icp', colors[0], 'No ICP'),
        # ('rgbd2colmap_colmap_poses', colors[1], 'RGBD + COLMAP Poses'),
        # ('colmap_reconstruction', colors[2], 'COLMAP'),
        ('rgbd2colmap', colors[3], 'With ICP')
    ],
    title="Peak Memory Usage",
    y_label="Memory (MB)",
    y_scale="log",
    y_tickvals=[250, 500, 1000, 2000],
    y_ticktext=['250', '500', '1000', '2000'],
    conversion_factor=1/1024,  # Convert KB to MB
    figure_size=(800, 500)
)
fig2.show()

rgbd2colmap_no_icp: 7.52 ± 2.33
rgbd2colmap: 67.63 ± 24.01


rgbd2colmap_no_icp: 2174.91 ± 937.12
rgbd2colmap: 2175.89 ± 937.49


In [ ]:
def generate_color_palette(n=4):
    """
    Generate a standard color palette with 4 distinct colors.
    
    Parameters:
    -----------
    n : int
        Number of colors to generate (default is 4)
    
    Returns:
    --------
    list
        List of color strings in rgb format
    """
    # Pre-defined colors that work well together
    colors = [
        'rgb(31, 119, 180)',   # blue
        'rgb(255, 127, 14)',   # orange
        'rgb(44, 160, 44)',    # green
        'rgb(214, 39, 40)'     # red
    ]
    
    # Add more colors if needed
    if n > 4:
        # Additional colors
        extra_colors = [
            'rgb(148, 103, 189)',  # purple
            'rgb(140, 86, 75)',    # brown
            'rgb(227, 119, 194)',  # pink
            'rgb(127, 127, 127)'   # gray
        ]
        colors.extend(extra_colors)
    
    # Return the requested number of colors
    return colors[:n]

def visualize_gaussian_counts(csv_filepath, colors=None):
    """
    Create a visualization of total Gaussians vs Steps for different voxel sizes.
    
    Parameters:
    -----------
    csv_filepath : str
        Path to the CSV file containing data with the format:
        "Step","office26_vs_003 - total_gaussians","office26_vs_003 - total_gaussians__MIN",...
    colors : list, optional
        List of colors to use for plotting. If None, will use the default color palette.
    
    Returns:
    --------
    tuple
        (fig, color_mapping) where:
        - fig is the plotly.graph_objects.Figure
        - color_mapping is a dict mapping voxel sizes to colors
    """
    # Read the CSV file
    df = pd.read_csv(csv_filepath)
    
    # Extract column names
    columns = df.columns.tolist()
    
    # Identify the step column and data columns (excluding MIN/MAX)
    step_col = "Step"
    data_cols = [col for col in columns if "total_gaussians" in col and "__MIN" not in col and "__MAX" not in col]
    
    # Create a figure
    fig = go.Figure()
    
    # Generate or use provided colors
    if colors is None:
        colors = generate_color_palette(len(data_cols))
    
    # Dictionary to map voxel sizes to colors
    color_mapping = {}
    
    voxel_sizes = {}
    # Add traces for each voxel size
    for i, col in enumerate(data_cols):
        # Extract scene name and voxel size from column name
        # Pattern: scene_vs_XXX - total_gaussians
        match = re.search(r'(\w+)_vs_(\d+)', col)
        if match:
            scene_name = match.group(1)
            voxel_size = match.group(2)
            # Format voxel size with leading zeros removed
            # Create label
            formatted_voxel_size = list(voxel_size)
            formatted_voxel_size.insert(1, '.')
            formatted_voxel_size = ''.join(formatted_voxel_size)
            label = f"voxel size = {formatted_voxel_size}"
            
            # Store color mapping
            key = formatted_voxel_size
            color_mapping[key] = colors[i % len(colors)]

            voxel_sizes[formatted_voxel_size] = (i, col, label)
        else:
            label = col
            key = col
            color_mapping[key] = colors[i % len(colors)]
        
    for voxel_size, (i, col, label) in sorted(voxel_sizes.items(), key=lambda x: float(x[0])):  # Sort by voxel size
        # Add trace
        fig.add_trace(go.Scatter(
            x=df[step_col],
            y=df[col],
            mode='lines',
            name=label,
            line=dict(color=colors[i % len(colors)], width=6)
        ))
    
    # Update layout
    fig.update_layout(
        font=dict(size=25),
        title="Total 3D Gaussians Amount",
        title_x=0.5,
        xaxis_title="Training Steps",
        xaxis_range=[0, 16000],
        yaxis_title="Gaussians Amount",
        legend=dict(
            yanchor='bottom',
            y=0.0,
            xanchor='right',
            x=1.0
        ),
        template="plotly_white",
        hovermode="x unified",
        width=600,
        height=600
    )

    
    return fig, color_mapping


csv_path = "../../artifacts/wandb_csvs/density_gs_count.csv"

# Optional: Define custom colors
custom_colors = [
    'rgba(31, 119, 180, 0.5)',   # blue
    'rgba(255, 127, 14, 0.5)',   # orange
    'rgba(44, 160, 44, 0.5)',    # green
    'rgba(214, 39, 40, 0.5)'     # red
]

# Generate visualization with custom colors
fig, color_mapping = visualize_gaussian_counts(csv_path, colors=custom_colors)

# The color_mapping can now be used in other visualization functions
print("Color mapping for other visualizations:", color_mapping)

fig.show()
fig.write_image("../../artifacts/figures/density_gaussians_count.pdf")

Color mapping for other visualizations: {'0.03': 'rgba(31, 119, 180, 0.5)', '0.3': 'rgba(255, 127, 14, 0.5)', '0.1': 'rgba(44, 160, 44, 0.5)', '0.05': 'rgba(214, 39, 40, 0.5)'}


In [ ]:
def visualize_training_times(
    csv_filepath: str,
    output_path: str = None,
    custom_colors: list = None,
    figure_size: tuple = (600, 600)
) -> go.Figure:
    """
    Create a bar plot visualization of training times for different voxel sizes.
    
    Parameters:
    -----------
    csv_filepath : str
        Path to the CSV file containing training time data
    output_path : str, optional
        Path to save the figure. If None, figure is not saved
    custom_colors : list, optional
        List of custom colors to use for the bars
    figure_size : tuple, optional
        (width, height) of the figure. Defaults to (800, 500)
    
    Returns:
    --------
    plotly.graph_objects.Figure
        The generated figure
    """
    # Read the CSV file
    df = pd.read_csv(csv_filepath)
    
    # Filter out MIN/MAX columns and get only training time columns
    time_cols = [col for col in df.columns 
                if "training_time_sec" in col 
                and not col.endswith(("__MIN", "__MAX"))]
    
    # Extract voxel sizes and times
    voxel_sizes = []
    times = []
    
    for col in time_cols:
        # Extract voxel size from column name (e.g., "office26_vs_003")
        match = re.search(r'_vs_(\d+)', col)
        if match:
            voxel_size = match.group(1)
            # Format voxel size with decimal point
            formatted_size = f"{voxel_size[0]}.{voxel_size[1:]}"
            voxel_sizes.append(formatted_size)
            times.append(df[col].iloc[0] / 60)  # Convert seconds to minutes
    
    # Sort by voxel size
    sorted_data = sorted(zip(voxel_sizes, times), key=lambda x: float(x[0]))
    voxel_sizes, times = zip(*sorted_data)
    
    # Set up colors
    if custom_colors is None:
        custom_colors = [
            "rgba(83, 135, 221, 1)",  # Blue
            "rgba(218, 76, 76, 1)",   # Red
            "rgba(34, 148, 135, 1)",  # Teal
            "rgba(237, 183, 50, 1)",  # Yellow
        ]
    
    # Create figure
    fig = go.Figure()
    
    # Add bars
    fig.add_trace(go.Bar(
        x=voxel_sizes,
        y=times,
        marker_color=[custom_colors[voxel_size] for voxel_size in voxel_sizes],
    ))
    
    # Update layout
    fig.update_layout(
        title={
            'text': "Training Time by Voxel Size",
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title="Voxel Size",
        yaxis_title="Training Time (minutes)",
        font=dict(size=25),
        template='plotly_white',
        width=figure_size[0],
        height=figure_size[1],
        showlegend=False,
        yaxis=dict(
            nticks=10,
            tickformat=".1f",
        )
    )
    
    # Save figure if output path is provided
    if output_path:
        fig.write_image(output_path)
    
    return fig



fig = visualize_training_times(
    csv_filepath="../../artifacts/wandb_csvs/density_gs_time.csv",
    output_path="../../artifacts/figures/density_training_times.pdf",
    custom_colors=color_mapping
)
fig.show()

In [ ]:
def plot_frames_performance(
    csv_path: str,
    output_path: str,
    data_key: str,
    pipelines: List[Tuple[str, str, str]] = None,
    title: str = None,
    y_label: str = None,
    x_label: str = "Number of Frames",
    y_scale: str = 'linear',
    y_tickvals: List[float] = None,
    y_ticktext: List[str] = None,
    figure_size: Tuple[Optional[int], Optional[int]] = (None, None),
    conversion_factor: float = 1.0,
    print_stats: bool = True,
    use_lines: bool = True,
    use_markers: bool = True,
    marker_size: int = 24,
    line_width: int = 6
):
    """
    Create a separate plot per pipeline, each saved as a PDF.
    """
    csv_path = Path(csv_path)
    output_path = Path(output_path)
    output_path.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(csv_path).drop(columns=["Step"], errors="ignore").astype(float)

    # Parse all relevant columns
    data_points = []
    for col in df.columns:
        if data_key in col and not col.endswith(("__MIN", "__MAX")):
            frame_match = re.search(r'frames(\d+)', col)
            pipeline_match = re.search(r'-([^-]+)-frames', col)
            if frame_match and pipeline_match:
                data_points.append({
                    'scene': col.split('-')[0],
                    'pipeline': pipeline_match.group(1),
                    'frames': int(frame_match.group(1)),
                    'value': df[col].iloc[0] * conversion_factor
                })

    data_df = pd.DataFrame(data_points)

    if pipelines is None:
        default_colors = ["rgba(83, 135, 221, 1)", "rgba(218, 76, 76, 1)",
                          "rgba(75, 192, 192, 1)", "rgba(153, 102, 255, 1)", 
                          "rgba(255, 159, 64, 1)"]
        unique_pipelines = data_df['pipeline'].unique()
        pipelines = [(p, default_colors[i % len(default_colors)], p) for i, p in enumerate(unique_pipelines)]

    mode = '+'.join(filter(None, ['lines' if use_lines else '', 'markers' if use_markers else '']))

    for pipeline_id, color, label in pipelines:
        pipeline_data = data_df[data_df['pipeline'] == pipeline_id]

        if pipeline_data.empty:
            print(f"Warning: No data found for pipeline '{pipeline_id}'")
            continue

        pipeline_data = pipeline_data.sort_values('frames')
        multiple_scenes = len(pipeline_data['scene'].unique()) > 1
        fig = go.Figure()

        if multiple_scenes:
            grouped = pipeline_data.groupby('frames')['value'].agg(['mean', 'std']).reset_index()
            fig.add_trace(go.Scatter(
                x=grouped['frames'],
                y=grouped['mean'],
                mode=mode,
                name=label,
                line=dict(color=color, width=line_width) if use_lines else None,
                marker=dict(size=marker_size, color=color) if use_markers else None,
                error_y=dict(type='data', array=grouped['std'], visible=True, color=color)
            ))
            if print_stats:
                for _, row in grouped.iterrows():
                    print(f"{pipeline_id} ({row['frames']} frames): {row['mean']:.2f} ± {row['std']:.2f}")
        else:
            fig.add_trace(go.Scatter(
                x=pipeline_data['frames'],
                y=pipeline_data['value'],
                mode=mode,
                name=label,
                line=dict(color=color, width=line_width) if use_lines else None,
                marker=dict(size=marker_size, color=color) if use_markers else None,
            ))
            if print_stats:
                for _, row in pipeline_data.iterrows():
                    print(f"{pipeline_id} ({row['frames']} frames): {row['value']:.2f}")

        final_title = title or f"{data_key.replace('_', ' ').title()} vs. Frame Count"
        final_y_label = y_label or data_key.replace('_', ' ').title()

        layout_args = dict(
            title=dict(text=final_title, x=0.5, xanchor='center'),
            xaxis_title=x_label,
            yaxis_title=final_y_label,
            font=dict(size=25),
            legend=dict(yanchor='top', y=0.98, xanchor='right', x=0.98),
            margin=dict(b=100),
            yaxis=dict(
                type=y_scale,
                tickangle=0,
                automargin=True,
                nticks=6 if y_tickvals is None else len(y_tickvals),
                tickvals=y_tickvals,
                ticktext=y_ticktext,
                range=[0, max(pipeline_data['value']) * 1.1]
            ),
            xaxis_tickvals=pipeline_data['frames'].tolist(),
            template='plotly_white'
        )

        if figure_size[0]:
            layout_args['width'] = figure_size[0]
        if figure_size[1]:
            layout_args['height'] = figure_size[1]

        fig.update_layout(**layout_args)

        save_path = output_path / f"{pipeline_id}_{data_key}.pdf"
        fig.write_image(str(save_path))
        print(f"Saved: {save_path}")
        fig.show()



fig1 = plot_frames_performance(
    csv_path="../../artifacts/wandb_csvs/frames_time.csv",
    output_path="../../artifacts/figures/frames",
    data_key="reconstruction_elapsed_time",
    pipelines=[
        ('rgbd2colmap_no_icp', colors[0], 'RGB-D (No ICP)'),
        ('colmap_reconstruction', colors[2], 'COLMAP')
    ],
    title="Reconstruction Time vs. Frame Count",
    y_label="Time (s)",
    # y_scale="log",
    # y_tickvals=[5, 10, 20, 50, 100, 200, 500],
    figure_size=(1000, 600)
)

# Example: Plot memory usage vs frame count
fig2 = plot_frames_performance(
    csv_path="../../artifacts/wandb_csvs/frames_memory.csv",
    output_path="../../artifacts/figures/frames",
    data_key="peak_memory_kb",
    pipelines=[
        ('rgbd2colmap_no_icp', colors[0], 'RGBD'),
        ('colmap_reconstruction', colors[2], 'COLMAP'),
    ],
    title="Peak Memory Usage vs. Frame Count",
    y_label="Memory (MB)",
    # y_scale="log",
    # y_tickvals=[100, 250, 500, 1000, 2000, 5000],
    conversion_factor=1/1024,  # Convert KB to MB
    figure_size=(1000, 600)
)

rgbd2colmap_no_icp (100 frames): 4.68
rgbd2colmap_no_icp (250 frames): 8.42
rgbd2colmap_no_icp (500 frames): 14.95
rgbd2colmap_no_icp (750 frames): 21.46
Saved: ../../artifacts/figures/frames/rgbd2colmap_no_icp_reconstruction_elapsed_time.pdf


colmap_reconstruction (100 frames): 81.98
colmap_reconstruction (250 frames): 214.77
colmap_reconstruction (500 frames): 417.54
colmap_reconstruction (750 frames): 1379.58
Saved: ../../artifacts/figures/frames/colmap_reconstruction_reconstruction_elapsed_time.pdf


rgbd2colmap_no_icp (100 frames): 1326.53
rgbd2colmap_no_icp (250 frames): 2872.12
rgbd2colmap_no_icp (500 frames): 5463.18
rgbd2colmap_no_icp (750 frames): 8071.62
Saved: ../../artifacts/figures/frames/rgbd2colmap_no_icp_peak_memory_kb.pdf


colmap_reconstruction (100 frames): 186.39
colmap_reconstruction (250 frames): 249.36
colmap_reconstruction (500 frames): 512.64
colmap_reconstruction (750 frames): 802.73
Saved: ../../artifacts/figures/frames/colmap_reconstruction_peak_memory_kb.pdf
